In [1]:
import pandas as pd 
import numpy as np 
import sklearn
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer



Importamos los datasets previamente transformados

In [2]:

#Primero ponemos la ruta
ruta_games = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\games.parquet"
ruta_items = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\users_items.parquet"
ruta_reviews = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\df_review_final.parquet"
#luego los extraemos
games = pd.read_parquet(ruta_games)
#items = pd.read_parquet(ruta_items)
reviews = pd.read_parquet(ruta_reviews)

In [3]:
# Primero se convertira la columna a int32 para un menor uso de memoria
reviews["Item_Id"] = reviews["Item_Id"].astype(float) 

# Ahora procederemos a unir los Datasets Games y Reviews

In [4]:


Data = pd.merge(games, reviews, how='inner', left_on='Id', right_on='Item_Id')
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49777 entries, 0 to 49776
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   App_Name              49777 non-null  object 
 1   Id                    49777 non-null  int32  
 2   Genres                49777 non-null  object 
 3   Developer             49777 non-null  object 
 4   Publisher             49777 non-null  object 
 5   Price                 49777 non-null  float64
 6   Release_Date          49713 non-null  object 
 7   User_Id               49777 non-null  object 
 8   Item_Id               49777 non-null  float64
 9   Recommend             49777 non-null  object 
 10  Analisis_Sentimiento  49777 non-null  int64  
dtypes: float64(2), int32(1), int64(1), object(7)
memory usage: 4.0+ MB


Procedemos a eliminar la columna Id ya que tiene los mismos datos que Item_Id

In [5]:

comparison = Data["Id"] == Data["Item_Id"]
value_counts = comparison.value_counts(normalize=True)
print(value_counts)


True    1.0
Name: proportion, dtype: float64


In [6]:
Data.drop(columns=["Id"], inplace = True)
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49777 entries, 0 to 49776
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   App_Name              49777 non-null  object 
 1   Genres                49777 non-null  object 
 2   Developer             49777 non-null  object 
 3   Publisher             49777 non-null  object 
 4   Price                 49777 non-null  float64
 5   Release_Date          49713 non-null  object 
 6   User_Id               49777 non-null  object 
 7   Item_Id               49777 non-null  float64
 8   Recommend             49777 non-null  object 
 9   Analisis_Sentimiento  49777 non-null  int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 3.8+ MB


In [7]:
# Vemos que hay nulos en la columna Release Date
Data.isnull().sum()
Data.dropna(inplace=True)

In [8]:
# Los eliminamos
Data.isnull().sum()

App_Name                0
Genres                  0
Developer               0
Publisher               0
Price                   0
Release_Date            0
User_Id                 0
Item_Id                 0
Recommend               0
Analisis_Sentimiento    0
dtype: int64

In [9]:
# Ahora vamos a sacar una muestra
muestra = 0.11
Data_sample = Data.sample(frac = muestra, random_state=42)
Data_sample.shape
# Ahora procederemos a resetear el index para conservar el orden
Data_sample = Data_sample.reset_index(drop=True)

In [10]:
Data_sample["User_Id"].unique()

array(['fukum008', '76561198071862339', '76561198051717662', ...,
       '76561198059969798', '76561198085982092', 'mozdawg'], dtype=object)

# Modelo de recomendacion

In [14]:
# Modelo de recomendacion usuario/item
def recomendacion_usuario(user_id):
    # Crear una instancia de TfidfVectorizer con stop words en inglés
    tfidf = TfidfVectorizer(stop_words='english')

    # Rellenar los valores nulos en la columna 'app_name' con una cadena vacía
    Data_sample['App_Name'] = Data_sample['App_Name'].fillna('')

    # Aplicar la transformación TF-IDF a los datos de la columna 'app_name'
    tfidf_matrix = tfidf.fit_transform(Data_sample['App_Name'])

    # Calcular la similitud coseno entre los juegos utilizando linear_kernel
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Obtener el índice del usuario específico en el DataFrame
    matching_users = Data_sample[Data_sample['User_Id'] == user_id]

    if not matching_users.empty:
        user_index = matching_users.index[0]

        # Obtener las recomendaciones basadas en similitud coseno y los filtros requeridos
        recommendations = []
        seen_games = set()  # Utilizar un conjunto para evitar duplicados
        for i, score in sorted(enumerate(cosine_sim[user_index]), key=lambda x: x[1], reverse=True):
            if Data_sample['Recommend'][i] and Data_sample['Analisis_Sentimiento'][i] in [0, 1, 2]:
                app_name = Data_sample['App_Name'][i]
                if app_name not in seen_games:
                    recommendations.append((app_name, score))
                    seen_games.add(app_name)

        # Seleccionar las primeras 5 recomendaciones
        top_recommendations = recommendations[:5]

        # Imprimir las recomendaciones
        print(f"Top 5 juegos recomendados para el usuario {user_id}:")
        for App_name, score in top_recommendations:
            print(f"{App_name} - Similaridad: {score}")
    else:
        print(f"No se encontró el usuario con ID: {user_id}")

Testeamos

In [15]:
recomendacion_usuario('mozdawg')

Top 5 juegos recomendados para el usuario mozdawg:
Call of Duty® 4: Modern Warfare® - Similaridad: 1.0
Call of Duty®: Modern Warfare® 2 - Similaridad: 1.0
Call of Duty®: Advanced Warfare - Gold Edition - Similaridad: 0.42703644877317937
Call of Duty®: Ghosts - Similaridad: 0.31485250167366635
Chivalry: Medieval Warfare - Similaridad: 0.2996532993444642


Exportamos a formato parquet

In [32]:
Data_sample.to_parquet("Data_Sample_Recommendation", engine = "pyarrow")